<a href="https://colab.research.google.com/github/eobianyor/Project3/blob/main/Model4IdahoFires_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [3]:
# IMPORT DEPENDENCIES
# FOR DATA
import pandas as pd
# import os
import csv
import math
import numpy as np
from numpy import logspace
import time
import datetime
# import requests
# import datefinder

# # FOR SQL LITE
# from sqlalchemy import create_engine
# from datetime import date

# # FOR PLOTTING
import matplotlib.pyplot as plt
from matplotlib import style
style.use("fivethirtyeight")
from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 8

# FOR MODELING
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# FOR KNN AND OTHER MODELS
from scipy.optimize import curve_fit
# from splinter import Browser
# from bs4 import BeautifulSoup as BS
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.datasets import make_blobs
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# # FOR TF MODELS
# import keras.models
# import keras.layers
# import keras.utils
# from keras.models import Sequential
# from keras.layers import Dense, Dropout

In [6]:
# IMPORT LIBRARIES FOR COLAB
# =========================================================
#               Python Libraries for Colab
# =========================================================
import sys
# ---------------------------------------------------------
from google.colab import drive

# =========================================================
#               Mount Google Drive to Colab
# =========================================================
drive.mount('/content/gdrive')


Mounted at /content/gdrive


IMPORT DATA

In [45]:
# File path for the CSV Files
# idahoFireWeatherDrought = os.path.join("/content/gdrive/My Drive/Colab Notebooks/IdahoFirePorject/fires_Idaho_2000_2015_drought_weather_Plus.csv")
# idahoFireWeatherDrought = "/content/gdrive/My Drive/Colab Notebooks/IdahoFireProject/fires_Idaho_2000_2015_drought_weather_Plus.csv"
idahoFireWeatherConditions = "/content/gdrive/My Drive/Colab Notebooks/IdahoFireProject/mnth_wthr_fire_strmflw_lghtnng_strk_mrgd_df_FINAL.csv"

# Open the CSV Files, Convert to a Dataframe, and Save as a Variable
idahoFireWeatherConditions_df = pd.read_csv(idahoFireWeatherConditions)


DATA CLEAN

In [46]:
idahoFireWeatherConditions_df

,Unnamed: 0,fire_true_no_fire_false,avg_temp,prcp,humidity,dew_point,strms_blw_extndd_wghtd_avg,number_of_strikes
0,0,False,30.082051,2.217854,78.800600,-4.567051,9.079257,0.0
1,1,False,33.382051,0.353707,71.178455,-4.291505,11.851520,0.0
2,2,False,42.292308,1.035585,58.164373,-3.043872,5.375434,0.0
3,3,False,45.738462,1.665829,57.112693,-1.460228,1.763397,0.0
4,4,True,59.920513,1.093634,52.850319,4.295041,0.563885,1.0
...,...,...,...,...,...,...,...,...
223,223,True,73.245455,0.379795,36.460167,5.492461,43.084198,11.0
224,224,False,63.113636,0.715909,43.361013,3.332986,44.911618,0.0
225,225,False,55.747727,1.084250,59.380000,4.525904,56.129587,0.0
226,226,False,34.452273,1.391068,71.776756,-2.996882,56.625477,0.0


EXTRACT X AND y FOR MODEL

In [9]:
# Create X Input by keeping columns from df
X = idahoFireWeatherConditions_df[['avg_temp', 'prcp', 'humidity', 'dew_point', 'strms_blw_extndd_wghtd_avg', 'number_of_strikes']]

# X by drop
# X = idaho_Fire_Weather_df.drop(['FIRE_SIZE_CLASS', 'DISCOVERY_DATE_CONVERTED'], axis=1)

# Get dummies for names (if using the county 'NAME' column)
# X = pd.get_dummies(X, columns=['NAME'])

print(X.keys())
X = X.values.reshape(-1, 6)

# X = X.to_numpy()

print(X.shape)
type(X)
X

Index(['avg_temp', 'prcp', 'humidity', 'dew_point',
       'strms_blw_extndd_wghtd_avg', 'number_of_strikes'],
      dtype='object')
(228, 6)


array([[30.08205128,  2.21785366, 78.8006002 , -4.56705067,  9.0792566 ,
         0.        ],
       [33.38205128,  0.35370732, 71.17845473, -4.29150548, 11.85152009,
         0.        ],
       [42.29230769,  1.03558537, 58.16437269, -3.04387214,  5.37543396,
         0.        ],
       ...,
       [55.74772727,  1.08425   , 59.38000044,  4.52590371, 56.12958656,
         0.        ],
       [34.45227273,  1.39106818, 71.7767561 , -2.99688228, 56.62547746,
         0.        ],
       [29.20227273,  1.59402273, 79.20327356, -4.24170056, 42.65254033,
         0.        ]])

In [10]:
### CREATE y VALUES
y = idahoFireWeatherConditions_df[['fire_true_no_fire_false']]

# NO OF OUTPUTS (FOR ML MODELING)
NoOfOutputs = 2

# y = y.values.reshape(-1, 2)

# print(y.shape)
# # type(y)
# y

# LABEL ENCODE Y
# Import required module
from sklearn.preprocessing import LabelEncoder

# LabelEncoder
le = LabelEncoder()

# Create an object of the label encoder class
labelencoder = LabelEncoder()

# apply "le.fit_transform"
old_y = y.apply(le.fit_transform)
y = old_y

# # Change the shape of y v1
new_y = np.array(old_y)
y = new_y.reshape(-1, 1) 

# View output
print(f"Shape of y is {y.shape}")
print(f"Type of y is {type(y)}")
y

Shape of y is (228, 1)
Type of y is <class 'numpy.ndarray'>


array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
    

FORMAT MODEL INPUTS

In [12]:
# Split data into train and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
# # Scale your data
# from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
# # y_scaler = StandardScaler().fit(y_train)

# # Create variables to hold the scaled train & test data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# # y_train_scaled = y_scaler.transform(y_train)
# # y_test_scaled = y_scaler.transform(y_test)

In [15]:
### Encode the categorical target variable to the necessary format for the model
from keras.utils import to_categorical

# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [16]:
# X Inputs
print(X_train.shape)
print(X_train_scaled.shape)
print(X_test.shape)
print(X_test_scaled.shape)

# y Inputs
print(y_train_categorical.shape)
print(y_test_categorical.shape)

(171, 6)
(171, 6)
(57, 6)
(57, 6)
(171, 2)
(57, 2)


MODELS

In [18]:
# Create the GridSearchCV model
model1 = SVC(kernel="rbf")
model1

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [26]:
# Create the GridSearchCV parameters

# 'gamma': [0.0000001, 0.0000005, 0.0000001, 0.0000005, 0.000001, 0.000005, 0.000001]

param_grid = {'C': [1, 5, 10, 15, 20],
              'degree': [-1, -0.9, -0.5, 0, 0.5],
              'gamma': [0.01, 0.05, 0.1,],
              'tol': [0.5, 1, 2]}

# param_grid = {'C': [15],
#               'degree': [-0.9],
#               'gamma': [0.05],
#               'tol': [1]}

cv = KFold(n_splits=5, shuffle=False, random_state=None)
gridModel = GridSearchCV(model1, param_grid, verbose=3, scoring='accuracy', cv=cv)

In [27]:
# Train the model with GridSearch
gridModel.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 225 candidates, totalling 1125 fits
[CV] C=1, degree=-1, gamma=0.01, tol=0.5 .............................
[CV] ... C=1, degree=-1, gamma=0.01, tol=0.5, score=nan, total=   0.0s
[CV] C=1, degree=-1, gamma=0.01, tol=0.5 .............................
[CV] ... C=1, degree=-1, gamma=0.01, tol=0.5, score=nan, total=   0.0s
[CV] C=1, degree=-1, gamma=0.01, tol=0.5 .............................
[CV] ... C=1, degree=-1, gamma=0.01, tol=0.5, score=nan, total=   0.0s
[CV] C=1, degree=-1, gamma=0.01, tol=0.5 .............................
[CV] ... C=1, degree=-1, gamma=0.01, tol=0.5, score=nan, total=   0.0s
[CV] C=1, degree=-1, gamma=0.01, tol=0.5 .............................
[CV] ... C=1, degree=-1, gamma=0.01, tol=0.5, score=nan, total=   0.0s
[CV] C=1, degree=-1, gamma=0.01, tol=1 ...............................
[CV] ..... C=1, degree=-1, gamma=0.01, tol=1, score=nan, total=   0.0s
[CV] C=1, degree=-1, gamma=0.01, tol=1 ...............................
[CV] ..... C=

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: degree of polynomial kernel < 0

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/

[CV]  C=1, degree=-0.9, gamma=0.1, tol=0.5, score=0.912, total=   0.0s
[CV] C=1, degree=-0.9, gamma=0.1, tol=0.5 ............................
[CV]  C=1, degree=-0.9, gamma=0.1, tol=0.5, score=0.912, total=   0.0s
[CV] C=1, degree=-0.9, gamma=0.1, tol=0.5 ............................
[CV]  C=1, degree=-0.9, gamma=0.1, tol=0.5, score=0.912, total=   0.0s
[CV] C=1, degree=-0.9, gamma=0.1, tol=0.5 ............................
[CV]  C=1, degree=-0.9, gamma=0.1, tol=0.5, score=0.912, total=   0.0s
[CV] C=1, degree=-0.9, gamma=0.1, tol=1 ..............................
[CV] .. C=1, degree=-0.9, gamma=0.1, tol=1, score=0.886, total=   0.0s
[CV] C=1, degree=-0.9, gamma=0.1, tol=1 ..............................
[CV] .. C=1, degree=-0.9, gamma=0.1, tol=1, score=0.941, total=   0.0s
[CV] C=1, degree=-0.9, gamma=0.1, tol=1 ..............................
[CV] .. C=1, degree=-0.9, gamma=0.1, tol=1, score=0.941, total=   0.0s
[CV] C=1, degree=-0.9, gamma=0.1, tol=1 ..............................
[CV] .

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

[CV] .. C=1, degree=0, gamma=0.01, tol=0.5, score=0.853, total=   0.0s
[CV] C=1, degree=0, gamma=0.01, tol=0.5 ..............................
[CV] .. C=1, degree=0, gamma=0.01, tol=0.5, score=0.882, total=   0.0s
[CV] C=1, degree=0, gamma=0.01, tol=1 ................................
[CV] .... C=1, degree=0, gamma=0.01, tol=1, score=0.971, total=   0.0s
[CV] C=1, degree=0, gamma=0.01, tol=1 ................................
[CV] .... C=1, degree=0, gamma=0.01, tol=1, score=0.912, total=   0.0s
[CV] C=1, degree=0, gamma=0.01, tol=1 ................................
[CV] .... C=1, degree=0, gamma=0.01, tol=1, score=0.912, total=   0.0s
[CV] C=1, degree=0, gamma=0.01, tol=1 ................................
[CV] .... C=1, degree=0, gamma=0.01, tol=1, score=0.824, total=   0.0s
[CV] C=1, degree=0, gamma=0.01, tol=1 ................................
[CV] .... C=1, degree=0, gamma=0.01, tol=1, score=0.882, total=   0.0s
[CV] C=1, degree=0, gamma=0.01, tol=2 ................................
[CV] .

...

[CV] . C=20, degree=0, gamma=0.01, tol=0.5, score=0.941, total=   0.0s
[CV] C=20, degree=0, gamma=0.01, tol=0.5 .............................
[CV] . C=20, degree=0, gamma=0.01, tol=0.5, score=0.941, total=   0.0s
[CV] C=20, degree=0, gamma=0.01, tol=0.5 .............................
[CV] . C=20, degree=0, gamma=0.01, tol=0.5, score=0.941, total=   0.0s
[CV] C=20, degree=0, gamma=0.01, tol=0.5 .............................
[CV] . C=20, degree=0, gamma=0.01, tol=0.5, score=0.941, total=   0.0s
[CV] C=20, degree=0, gamma=0.01, tol=1 ...............................
[CV] ... C=20, degree=0, gamma=0.01, tol=1, score=0.914, total=   0.0s
[CV] C=20, degree=0, gamma=0.01, tol=1 ...............................
[CV] ... C=20, degree=0, gamma=0.01, tol=1, score=0.912, total=   0.0s
[CV] C=20, degree=0, gamma=0.01, tol=1 ...............................
[CV] ... C=20, degree=0, gamma=0.01, tol=1, score=0.941, total=   0.0s
[CV] C=20, degree=0, gamma=0.01, tol=1 ...............................
[CV] .

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

[CV] . C=20, degree=0.5, gamma=0.05, tol=2, score=0.941, total=   0.0s
[CV] C=20, degree=0.5, gamma=0.05, tol=2 .............................
[CV] . C=20, degree=0.5, gamma=0.05, tol=2, score=0.912, total=   0.0s
[CV] C=20, degree=0.5, gamma=0.05, tol=2 .............................
[CV] . C=20, degree=0.5, gamma=0.05, tol=2, score=0.912, total=   0.0s
[CV] C=20, degree=0.5, gamma=0.05, tol=2 .............................
[CV] . C=20, degree=0.5, gamma=0.05, tol=2, score=0.971, total=   0.0s
[CV] C=20, degree=0.5, gamma=0.1, tol=0.5 ............................
[CV]  C=20, degree=0.5, gamma=0.1, tol=0.5, score=0.971, total=   0.0s
[CV] C=20, degree=0.5, gamma=0.1, tol=0.5 ............................
[CV]  C=20, degree=0.5, gamma=0.1, tol=0.5, score=0.971, total=   0.0s
[CV] C=20, degree=0.5, gamma=0.1, tol=0.5 ............................
[CV]  C=20, degree=0.5, gamma=0.1, tol=0.5, score=0.971, total=   0.0s
[CV] C=20, degree=0.5, gamma=0.1, tol=0.5 ............................
[CV]  

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10, 15, 20],
                         'degree': [-1, -0.9, -0.5, 0, 0.5],
                         'gamma': [0.01, 0.05, 0.1], 'tol': [0.5, 1, 2]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [29]:
# List best parameters and best score
print(gridModel.best_params_)
print(gridModel.best_score_)

{'C': 15, 'degree': -0.9, 'gamma': 0.05, 'tol': 1}
0.9648739495798321


In [30]:
# Testing the model
print(f"Training Data Score: {gridModel.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {gridModel.score(X_test_scaled, y_test)}")

Training Data Score: 0.9707602339181286
Testing Data Score: 0.8596491228070176


In [31]:
# Make predictions with the hypertuned model
predictions = gridModel.predict(X_test)
y_test_df = y_test.ravel()

prediction_df = pd.DataFrame({'predicted_values': predictions,
                   'Values_from_data': y_test_df})
prediction_df.head(60)

,predicted_values,Values_from_data
0,0,0
1,0,0
2,0,0
3,0,0
4,0,1
5,0,0
6,0,0
7,0,0
8,0,0
9,0,1


In [36]:
# --------------------------------------------------------------- #
# SAVE MODEL
# --------------------------------------------------------------- #
# SAVE MODEL via joblib
# --------------------------------------------------------------- #
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
# filename = 'gridModel_model2_v1.sav'
filename = '/content/gdrive/My Drive/Colab Notebooks/IdahoFireProject/gridModel_model2_v1.sav'
joblib.dump(gridModel, filename)

# SAVE MODEL via pickle
# --------------------------------------------------------------- #
import pickle

# with open('gridModel_model2_v1.pkl','wb') as modelFile:
with open('/content/gdrive/My Drive/Colab Notebooks/IdahoFireProject/gridModel_model2_v1.pkl','wb') as modelFile:
    pickle.dump(gridModel, modelFile)

In [38]:
# TEST MODEL WITH NEW DATA
# --------------------------------------------------------------- #
# LOAD MODEL via joblib
# --------------------------------------------------------------- #

# model_path = os.path.join("gridModel_model2_v1.sav")
# reloadedgridModel = joblib.load(model_path, mmap_mode='r')

# LOAD MODEL via pickle
# --------------------------------------------------------------- #

with open('/content/gdrive/My Drive/Colab Notebooks/IdahoFireProject/gridModel_model2_v1.pkl', 'rb') as modelReload:
    reloadedgridModel = pickle.load(modelReload)

In [57]:
# IMPORT CSV FOR DF
# --------------------------------------------------------------- #
X_input_Demo = "/content/gdrive/My Drive/Colab Notebooks/IdahoFireProject/Demo_Input.csv"


# Open the CSV Files, Convert to a Dataframe, and Save as a Variable
X_input_Demo_df = pd.read_csv(X_input_Demo)

# PREP X VALUES FOR MODEL
# --------------------------------------------------------------- #
# # Reshape X from df to array v1
newInput = X_input_Demo_df.values.reshape(-1, 6)

# # # View output
print(f"Shape of X Input is {newInput.shape}")
print(f"Type of X input is {type(newInput)}")

# RUN TEST
# --------------------------------------------------------------- #
print(f"Model output: {np.argmax(reloadedgridModel.predict(newInput), axis=1)}")
np.argmax(New_model.predict(new_data))

# # CREATE TO VIEW PREDICTIONS
# # --------------------------------------------------------------- #
# predictions_newInput = np.argmax(reloadedgridModel.predict(newInput), axis=1)
# probs = reloadedgridModel.predict(newInput)

# pred_df = pd.DataFrame({
#     "Predictions": predictions_newInput,
#     "P(0) model": np.round(probs[:, 0], 5),
#     "P(100) model": np.round(probs[:, 1], 5),
#     })

# pred_df
# print(X_input_dfFormat_df['D1'])

Shape of X Input is (5, 6)
Type of X input is <class 'numpy.ndarray'>


AxisError: ignored